# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [81]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()


In [82]:

# Part 1
from random import shuffle
from Utils import train_test_validation
# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)

X_train, X_test, X_valid, Y_train, Y_test, Y_valid = train_test_validation(X, Y, test_size = 0.15, validation_size= 0.25, shuffle = False)

print(X_train.shape, X_test.shape, X_valid.shape, Y_train.shape, Y_test.shape, Y_valid.shape)

print(np.abs(np.max(X_valid)))

(6709, 6) (1677, 6) (2797, 6) (6709,) (1677,) (2797,)
20.686865


In [83]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Create a pipeline with StandardScaler and LogisticRegression
model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression()) 
])

model.fit(X_train, Y_train)

y_pred = model.predict(X_test)




In [84]:

# Part 3

# Compute precision and recall on the test set using 
from zmq import NULL
from Utils import classification_report_interval

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)
report = classification_report_interval(Y_test, y_pred, alpha=0.05)
print(report)
precision0 = (0.94,1.00)
recall0 = (0.96, 1.00)
precision1 = (0.33, 100)
recall1 = (0.09,0.62)

#The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

            labels           precision             recall

                 0  0.98 : [0.94,1.00] 1.00 : [0.96,1.00]
                 1  0.70 : [0.33,1.00] 0.36 : [0.09,0.62]

          accuracy                                        0.98 : [0.94,1.00]



In [85]:

# Part 4

def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    cost = 0
    for i in range(len(predictions)):
        if Y[i] == 1 and predictions[i] == 0:
            cost += 30
        elif Y[i] == 0 and predictions[i] == 1:
            cost += 5
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    return cost/len(predictions)

In [86]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set
threshold = 2 

naive_cost = cost(model, threshold, X_test, Y_test)
print(naive_cost)

0.6976744186046512


In [87]:

# Part 5
thresholds = np.linspace(0,1,100)
min_cost = float('inf')
best_threshold = None
for threshold in thresholds:
    current_cost = cost(model, threshold, X_test, Y_test)
    if current_cost < min_cost:
        min_cost = current_cost
        best_threshold = threshold
optimal_threshold = best_threshold
cost_at_optimal_threshold = min_cost
print(optimal_threshold, cost_at_optimal_threshold)

0.11111111111111112 0.3518187239117472


In [88]:

# Part 6
# Part 6
from Utils import epsilon_bounded
from numpy import mean

cost_at_optimal_threshold_validation = cost(model, optimal_threshold, X_valid, Y_valid)
n = len(Y_valid)

b = 30
alpha = 0.05
epsilon = epsilon_bounded(n, b, alpha)
cost_interval = (cost_at_optimal_threshold_validation - epsilon, cost_at_optimal_threshold_validation + epsilon)

if cost_interval[0] < 0:
    cost_interval = (0, cost_interval[1])

print(cost_interval)
# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)



(0, 1.0617676271684535)


In [89]:

# Part 7
t = optimal_threshold #optimal threshold
f = model # The model
Y_valid2d = Y_valid.reshape(-1, 1) # reshape Y_valid to 2d array
X_valid.reshape(-1, 1) # reshape X_valid to 2d array

#C = 30 *(1 - (f.predict_proba(X_valid)[:,1] >= t)) * Y_valid + 5 * (1 - Y_valid) * (f.predict_proba(X_valid)[:,1] >= t)

C = np.array([cost(f, t, x.reshape(1,-1), y) for x, y in zip(X_valid, Y_valid2d)])

variance_of_C = np.var(C)
print(variance_of_C)

6.198613637958541


In [90]:

# Part 7
from Utils import bennett_epsilon

# A 95% confidence interval of the random variable C using Bennett's inequality
n = len(Y_valid)
sigma = np.sqrt(variance_of_C)
b = np.var(np.where(Y_valid == 1, 30, 5)) 
epsilon = bennett_epsilon(n=n,b=b, sigma=sigma, alpha=0.05)
interval_of_C = (cost_at_optimal_threshold_validation - epsilon, cost_at_optimal_threshold_validation + epsilon)

print(interval_of_C)
assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

Numerical error -2.2863655413374318e-15
(0.1573301344540109, 0.42543711617165947)
